# 备忘录模式

**备忘录模式(Memento Pattern):** 在不破坏封装的前提下，捕获一个对象的内部状态，并在该对象之外保存这个状态，这样可以在以后将对象恢复到原先保存的状态

**NOTE:**
- 在设计备忘录类时需要考虑其封装性，除了`Originator`类，不允许其他类来调用备忘录类`Memento`的构造函数与相关方法

**角色:**
- Originator(原发器): 普通类, 可以生成一个备忘录, 也可以从备忘录恢复状态
- Memento(备忘录): 用于存储原发器的内部状态
- Caretaker(负责人): 只负责保存备忘录, 但是不能/不应该修改备忘录

## 象棋悔棋实现

In [23]:
# 棋子
class Chessman(object):
    def __init__(self, label, x, y):
        self.label = label
        self.x = x
        self.y = y

    def display(self):
        print '%s 位于第 %s 行 第 %s 列' % (self.label, self.y + 1, self.x + 1)

    def save(self):
        return ChessmanMemento(self.label, self.x, self.y)

    def restore(self, m):
        self.label = m.label
        self.x = m.x
        self.y = m.y


# 棋子备忘录
class ChessmanMemento():
    def __init__(self, label, x, y):
        self._label = label
        self._x = x
        self._y = y

    @property
    def label(self):
        return self._label

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y


# 备忘录管理者: 负责人
class MementoCaretaker(object):
    def __init__(self):
        self._mementos = []  # 记录多个历史版本
        self._idx = len(self._mementos) - 1  # 历史版本指针

    def undo(self):
        self._idx -= 1
        if self._idx < 0:
            raise ValueError('No more histories')
        return self._mementos[self._idx]

    def redo(self):
        self._idx += 1
        if self._idx >= len(self._mementos) - 1:
            raise ValueError('No more redos')
        return self._mementos[self._idx]

    def add(self, m):
        if self._mementos:
            # 删除回溯过的版本
            self._mementos = self._mementos[:self._idx + 1]
        self._mementos.append(m)
        self._idx = len(self._mementos) - 1

In [33]:
taker = MementoCaretaker()
chessman = Chessman('车', 0, 0)
chessman.display()

taker.add(chessman.save())

chessman.x += 1
chessman.display()

taker.add(chessman.save())

chessman.y += 5
chessman.display()

taker.add(chessman.save())

print '-------undo-------'
chessman.restore(taker.undo())
chessman.display()
print '-------undo-------'
chessman.restore(taker.undo())
chessman.display()
print '-------redo-------'
chessman.restore(taker.redo())
chessman.display()

chessman.y += 3
chessman.display()

taker.add(chessman.save())

print '-------undo-------'
chessman.restore(taker.undo())
chessman.display()
print '-------undo-------'
chessman.restore(taker.undo())
chessman.display()

车 位于第 1 行 第 1 列
车 位于第 1 行 第 2 列
车 位于第 6 行 第 2 列
-------undo-------
车 位于第 1 行 第 2 列
-------undo-------
车 位于第 1 行 第 1 列
-------redo-------
车 位于第 1 行 第 2 列
车 位于第 4 行 第 2 列
-------undo-------
车 位于第 1 行 第 2 列
-------undo-------
车 位于第 1 行 第 1 列


In [34]:
chessman.restore(taker.undo())

ValueError: No more histories